In [1]:
%run cv.ipynb

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)
Using TensorFlow backend.


In [2]:
train_texts_features = pd.read_pickle(output_path + '/train_text_features9')
test_texts_features = pd.read_pickle(output_path + '/test_text_features9')

In [3]:
train_texts_features['positivity'] = pd.read_pickle(output_path + '/positivity_train2').positivity
test_texts_features['positivity'] = pd.read_pickle(output_path + '/positivity_test2').positivity

In [4]:
train_texts_features.drop('embedding', 1, inplace=True)
test_texts_features.drop('embedding', 1, inplace=True)
#train_texts_features['embedding_lgb'] = pd.read_pickle(output_path + '/train_30_2_svd').embeddings
#test_texts_features['embedding_lgb'] = pd.read_pickle(output_path + '/test_30_2_svd').embeddings
train_texts_features['embedding_keras'] = pd.read_pickle(output_path + '/train_80_1_svd').embeddings
test_texts_features['embedding_keras'] = pd.read_pickle(output_path + '/test_80_1_svd').embeddings

In [101]:
train_texts_features['embedding'] = pd.read_pickle(output_path + '/train_100_embs').embeddings
test_texts_features['embedding'] = pd.read_pickle(output_path + '/test_100_embs').embeddings

In [5]:
assigned_clusters3_15 = pd.read_pickle(output_path + '/assigned_clusters3_15').assigned_clusters3_15
train_texts_features['assigned_clusters'] = assigned_clusters3_15[0:train_texts_features.shape[0]]

In [6]:
train_data = pd.read_pickle(output_path + '/train_data')
test_data = pd.read_pickle(output_path + '/test_data')

In [7]:
stat = pd.concat((train_data, test_data), sort=False)[['metadata_ownerId', 'objectId']] \
    .groupby('metadata_ownerId').agg({'objectId': 'count'})
stat.rename(columns={'objectId': 'owner_count'}, inplace=True)

In [8]:
stat3 = pd.concat((train_data, test_data), sort=False)[['objectId', 'instanceId_userId']] \
    .groupby('instanceId_userId').agg({'objectId': 'count'})
stat3.rename(columns={'objectId': 'user_count'}, inplace=True)

In [9]:
#stat2 = pd.concat((train_data, test_data), sort=False)[['objectId', 'instanceId_userId']] \
#    .groupby('objectId').agg({'instanceId_userId': 'count'})
#stat2.rename(columns={'instanceId_userId': 'object_count'}, inplace=True)

In [10]:
train_data = train_data.join(stat, how='left', on='metadata_ownerId')
#train_data = train_data.join(stat2, how='left', on='metadata_ownerId')
train_data = train_data.join(stat3, how='left', on='instanceId_userId')
train_data = train_data.join(train_texts_features.set_index('objectId'), how='inner', on='objectId')

In [11]:
test_data = test_data.join(stat, how='left', on='metadata_ownerId')
#test_data = test_data.join(stat2, how='left', on='metadata_ownerId')
test_data = test_data.join(stat3, how='left', on='instanceId_userId')
test_data = test_data.join(test_texts_features.set_index('objectId'), how='inner', on='objectId')

In [12]:
qq = train_data[['instanceId_userId', 'objectId', 'audit_timestamp']].groupby(['instanceId_userId', 'objectId']).count()
qq.rename(columns={'audit_timestamp': 'edits_count'}, inplace=True)
train_data = train_data.join(qq, how='left', on=['instanceId_userId', 'objectId'])


In [13]:
qq = test_data[['instanceId_userId', 'objectId', 'audit_timestamp']].groupby(['instanceId_userId', 'objectId']).count()
qq.rename(columns={'audit_timestamp': 'edits_count'}, inplace=True)
test_data = test_data.join(qq, how='left', on=['instanceId_userId', 'objectId'])



In [14]:
class LgbModel(MyModel):
    def get_X(self, data):
        X = super().get_X(data)
        ###!!!!!!
        X.drop([c for c in X.columns if c.startswith('embedding') and not c.startswith('embedding_keras')], 1, inplace=True)
        X.drop(['end', 'is_ru'], 1, inplace=True)
        return X
    def fit(self, data):
        #data = data.drop_duplicates(['instanceId_userId', 'objectId'], keep='last')
        
        #owner_stat = data[['metadata_ownerId', 'label']].groupby('metadata_ownerId').mean()
        #owner_stat.rename(columns={'label': 'owner_mean_label'}, inplace=True)
        #self.owner_stat = owner_stat
        #data = data.join(owner_stat, how='left', on='metadata_ownerId')
        '''
        grp = data[['instanceId_userId', 'label']].groupby('instanceId_userId')
        self.owner_stat = grp.sum() / (grp.count() + 1)
        self.owner_stat.rename(columns={'label': 'owner_mean_label'}, inplace=True)

        owner_sum_stat = grp.sum()
        owner_cnt_stat = grp.count()
        owner_sum_stat.rename(columns={'label': 'owner_sum_label'}, inplace=True)
        owner_cnt_stat.rename(columns={'label': 'owner_cnt_label'}, inplace=True)

        data = data.join(owner_sum_stat, how='left', on='instanceId_userId')
        data = data.join(owner_cnt_stat, how='left', on='instanceId_userId')
        data['owner_mean_label'] = (data['owner_sum_label'] - data['label']) / data['owner_cnt_label']
        data.drop(['owner_sum_label', 'owner_cnt_label'], 1, inplace=True)
        '''
        y = data['label']
        lgb_train = lgb.Dataset(self.get_X(data), y)
        categorical_feature=['assigned_clusters']
        
        #if self.verbose >= 2:
        print('Starting train: %s' % datetime.datetime.now())
        params = self.params.copy()
        num_boost_round = params['num_boost_round']
        del params['num_boost_round']
        #params['objective'] = 'binary'
        params['metric'] = 'auc'
        self.model = lgb.train(
            params,
            lgb_train,
            num_boost_round=num_boost_round
        )
    def predict(self, data):
        #data = data.join(self.owner_stat, how='left', on='metadata_ownerId')
        #data = data.join(self.owner_stat, how='left', on='instanceId_userId')
        proba = self.model.predict(self.get_X(data))
        return proba

class KerasModel(MyModel):
    def get_X(self, data):
        X = super().get_X(data)
        X.drop([c for c in X.columns if c.startswith('embedding') and not c.startswith(self.params['emb'])], 1, inplace=True)
        X.drop(['end', 'is_ru'], 1, inplace=True)
        return X
    def fit(self, data):
        params = self.params
        self.scaler = MyScaler()
        X = self.get_X(data)
        self.scaler.fit_transform(X, inplace=True)
        y = data['label'].values
        
        model = keras.models.Sequential()
        self.model = model

        model.add(keras.layers.Dense(params['n1'], activation = "relu", input_shape=(X.shape[1], )))
        model.add(keras.layers.BatchNormalization())
        model.add(keras.layers.LeakyReLU())

        model.add(keras.layers.Dropout(params['dropout'], noise_shape=None, seed=1))
        model.add(keras.layers.Dense(params['n2'], activation = "relu"))
        model.add(keras.layers.BatchNormalization())
        
        if 'n3' in params:
            model.add(keras.layers.Dropout(params['dropout'], noise_shape=None, seed=1))
            model.add(keras.layers.Dense(params['n3'], activation = "relu"))
            model.add(keras.layers.BatchNormalization())

        model.add(keras.layers.Dense(1, activation = "sigmoid"))
        model.summary()
        model.compile(
            optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
            loss = "binary_crossentropy",
            metrics = ["accuracy"]
        )
        results = model.fit(
            X, y,
            epochs=params['epochs'],
            batch_size=params.get('batch_size', 1024),
            verbose=params['verbose']
        )

    def predict(self, X):
        X = self.get_X(X)
        self.scaler.transform(X, inplace=True)
        return self.model.predict(X)[:,0]


In [15]:
def feat(x):
    n = x.shape[0]
    return np.add.reduce(x.embedding.values) / n

class MyScaler():
    def fit_transform(self, X, inplace=False):
        assert inplace

        self.columns = []
        for c in X.columns:
            if c != 'label':
                self.columns.append(c)
        
        self.means = []
        self.sds = []
        self.mins = []
        self.maxs = []
        for c in self.columns:
            self.means.append(np.mean(X[c]))
            self.sds.append(np.std(X[c]))
            self.mins.append(np.min(X[c]))
            self.maxs.append(np.max(X[c]))
        return self.transform(X, inplace=inplace)
    
    def transform(self, X, inplace=False):
        assert inplace
        for c, mean, sd, mn, mx in zip(self.columns, self.means, self.sds, self.mins, self.maxs):
            X[c] = ((X[c] - mean) / sd).astype(np.float32)
            #X[c] = ((X[c].astype(np.float64) - float(mn)) / (float(mx) - float(mn))).astype(np.float32)
        
def create_features(data):
    ones = np.repeat(1, data.shape[0])
    res = pd.DataFrame({
        'instanceId_userId': data['instanceId_userId'],
        'objectId': data['objectId'],
        'object_type': data['instanceId_objectType'],
        'client_type': data['audit_clientType'],
#        'object_type_0': data['instanceId_objectType'] == 0,
#        'object_type_1': data['instanceId_objectType'] == 1,
#        'object_type_2': data['instanceId_objectType'] == 2,
#        'client_type_0': data['audit_clientType'] == 0,
#        'client_type_1': data['audit_clientType'] == 1,
#        'client_type_2': data['audit_clientType'] == 2,
        
        #'is_longread': data['len'] > 400,
        #'len': data['len'],
        #'p_len': data['p_len'],
        'len_log': np.log(data['len'] + 1).astype(np.float32),
        'p_len_log': np.log(data['p_len'] + 1).astype(np.float32),
        #'plen_per_len': data['p_len'] / data['len'],
        'q_count': np.log(data['q_count'] + 1).astype(np.float32),
        'links_count': np.log(data['links_count'] + 1).astype(np.float32),
        'emojis_rate': (data['emojis_count'] / data['len']).astype(np.float32),
        'upper_rate': (data['upper_count'] / data['len']).astype(np.float32),
        #'upper_count': data['upper_count'],
        'ok_videos_count': data['ok_videos_count'].clip(upper=1).astype(np.int8),
        'ok_groups_count': data['ok_groups_count'].clip(upper=1).astype(np.int8),
        'youtube_count': data['youtube_count'].clip(upper=1).astype(np.int8),
        #'is_video': (data['youtube_count'] + data['ok_videos_count']).clip(upper=1),
        'is_adv': data['is_adv'].astype(np.int8),
        'is_recipe': data['is_recipe'].astype(np.int8),
  #      'brackets_balance': data['brackets_balance'],
        'brackets_balance_log': np.log(data['brackets_balance'].abs() + 1).astype(np.float32),
        'quotes_count': np.log(data['quotes_count'] + 1).astype(np.float32),
        'mdots_count': np.log(data['mdots_count'] + 1).astype(np.float32),
        'e_count': np.log(data['e_count'] + 1).astype(np.float32),
        #'assigned_clusters': data['assigned_clusters'],
        'owner_count': data['owner_count'],
        'old': data['audit_timestamp'] - data['metadata_createdAt'],
        #'wtf': data['audit_timestamp'] <= data['metadata_createdAt'],
        #'mean_target': data['mean_target'],
        'has_phone': data['has_phone'].astype(np.int8),                                     #sm+
        'hashes_count': np.log(data['hashes_count'] + 1).astype(np.float32),#---
        'is_ru': data['lang'] == 'ru',#---
        ###'is_day': data['is_day'],#---
        'has_newline': data['lines_count'].clip(upper=1).astype(np.int8),                   #sm+
        #'object_count': data['object_count'],
        'user_count': data['user_count'],
        'end': data['e_end'] | data['q_end'] | data['d_end'],#---
        ####'md_count': data['md_count'].clip(upper=1),#---
        #'positivity': data['positivity'].fillna(0.5).astype(np.float32),
        'metadata_ownerId': data['metadata_ownerId'],
        #'unliked': data['unliked'],
        #'audit_timestamp_day': data['audit_timestamp'] % (3600*24),
        #'edits_count': data['edits_count'].clip(upper=2).astype(np.int8),
        'edits_count': np.log(data['edits_count'] + 1).astype(np.float32),
    })
    #qq = data[['instanceId_userId', 'embedding']].join(
    #        data.groupby('instanceId_userId').apply(feat).reset_index().set_index('instanceId_userId'), 
    #        on='instanceId_userId')
    #print('Cosing:')
    #res['emb_dist'] = (qq.iloc[:,1] - qq.iloc[:,2]).apply(np.linalg.norm)
    #res['emb_cosine_dist'] = qq.iloc[:,1:3].apply(lambda x: scipy.spatial.distance.cosine(*x), axis=1)
    #print('Embs:')
    
    #for e in ('embedding_lgb', 'embedding_keras', 'embedding'):
    for e in ('embedding_keras',):
    #for e in ('embedding',):
        emb = np.stack(data[e])
        for j in range(emb.shape[1]):
            res['%s_%d' % (e, j)] = emb[:,j].astype(np.float32)

    if 'liked' in data.columns:
        res['label'] = data['liked']
    for c in ('clicked', 'viewed', 'disliked', 'reshared', 'ignored', 'commented', 'complaint', 'unliked'):
        if c in data.columns:
            res[c] = data[c]
            
    return res

In [16]:
%%time
ttt = create_features(train_data.head(10000))

CPU times: user 94.7 ms, sys: 95.7 ms, total: 190 ms
Wall time: 352 ms


In [17]:
%%time
train = create_features(train_data)

CPU times: user 1min 18s, sys: 46.5 s, total: 2min 4s
Wall time: 2min 34s


In [ ]:
sub = train[train.instanceId_userId % 4 == 1]

In [48]:
sub = create_features(train_data[train_data.instanceId_userId % 4 == 1])

In [49]:
%%time
seed_everything(341)
cross_validation(MeanModel([KerasModel({
    'epochs': 10,
    'verbose': 2,
    'n1': 170,
    'n2': 170,
    'dropout': 0.1,
    'emb': 'embedding_keras',
}), LgbModel({
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 25,
    'lambda_l2': 0.0,
    'num_leaves': 30,
    'learning_rate': 0.35,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 1000,
    'verbose': 0
})], [0.5, 0.5]), sub, n_iters=1, verbose=2)


KFold(n_splits=5, random_state=2707, shuffle=True)
Prepare data: 2019-03-15 13:36:20.121715
Fit: 2019-03-15 13:36:30.404223
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 170)               17850     
_________________________________________________________________
batch_normalization_1 (Batch (None, 170)               680       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 170)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 170)               0         
_________________________________________________________________
dense_2 (Dense)              

KeyboardInterrupt: 

In [18]:
test = create_features(test_data)

In [ ]:
del test_data
del train_data
del train_texts_features
del test_texts_features
del sub
gc.collect()

In [21]:
%%time
seed_everything(341)
model3 = MeanModel([KerasModel({
    'epochs': 30,
    'verbose': 2,
    'n1': 170,
    'n2': 170,
    'dropout': 0.1,
    'emb': 'embedding_keras',
}), LgbModel({
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 25,
    'lambda_l2': 0.0,
    'num_leaves': 30,
    'learning_rate': 0.35,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 1000,
    'verbose': 0
})], [0.5, 0.5])

model3.fit(train)
pred3 = model3.predict(test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 170)               17850     
_________________________________________________________________
batch_normalization_1 (Batch (None, 170)               680       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 170)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 170)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 170)               29070     
_________________________________________________________________
batch_normalization_2

In [22]:
model3.coefs = [0.5, 0.5]
pred3 = model3.predict(test)

In [27]:
predT = pd.read_pickle(output_path + '/pred').T1.values

In [29]:
submitA3 = predict_to_submit(test, pred3 + predT * 0.2)
submitA3.to_csv(output_path + "/textSubmitA3.csv.gz", header=False, compression='gzip')

In [ ]:
%%time
seed_everything(341)

modelT = KerasModel({
    'epochs': 30,
    'verbose': 2,
    'n1': 170,
    'n2': 170,
    'n3': 85,
    'dropout': 0.15,
    'emb': 'embedding',
})
modelT.fit(train)

In [132]:
predT

array([0.12653881, 0.11429563, 0.17078048, ..., 0.05660952, 0.1017186 ,
       0.16263291], dtype=float32)

In [136]:
submitA1 = predict_to_submit(test, pred2 + predT * 0.2)
submitA1.to_csv(output_path + "/textSubmitA2.csv.gz", header=False, compression='gzip')

In [137]:
pd.DataFrame({'T1': predT, 'pred2': pred2, 'pred': pred}).to_pickle(output_path + '/pred')

In [21]:
%%time
seed_everything(341)
'''
KerasModel({
    'epochs': 30,
    'verbose': 2,
    'n1': 170,
    'n2': 170,
    'n3': 85,
    'dropout': 0.15,
    'emb': 'embedding',
}), 
'''
model = MeanModel([KerasModel({
    'epochs': 30,
    'verbose': 2,
    'n1': 170,
    'n2': 170,
    'dropout': 0.1,
    'emb': 'embedding_keras',
}), LgbModel({
    'boosting_type': 'gbdt',
    'min_data_in_leaf': 25,
    'lambda_l2': 0.0,
    'num_leaves': 30,
    'learning_rate': 0.35,
    'feature_fraction': 1,
    'bagging_fraction': 1,
    'bagging_freq': 5,
    'num_boost_round': 1000,
    'verbose': 0
})], [0.5, 0.5])

model.fit(train)
pred = model.predict(test)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 170)               18190     
_________________________________________________________________
batch_normalization_1 (Batch (None, 170)               680       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 170)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 170)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 170)               29070     
_________________________________________________________________
batch_normalization_2

In [22]:
submit = predict_to_submit(test, pred)
submit.to_csv(output_path + "/textSubmit1.csv.gz", header=False, compression='gzip')

In [116]:
lens_stat = defaultdict(int)
for r in submit:
    lens_stat[len(r)] += 1
lens_stat = pd.DataFrame([(k, v) for k, v in lens_stat.items()], columns=['len', 'count'])
lens_stat['relative_count'] = lens_stat['count'] / lens_stat['count'].sum()
lens_stat.head()

,len,count,relative_count
0,2,66422,0.318806
1,3,43664,0.209574
2,4,27742,0.133154
3,8,6431,0.030867
4,16,913,0.004382


In [146]:
lang_stat = pd.concat((train_data, test_data), sort=False)[['lang', 'instanceId_userId']].head(200000) \
    .groupby('lang').count().reset_index().sort_values(['instanceId_userId']).reset_index()
lang_stat.rename(columns={'index': 'lang_index'}, inplace=True)
lang_stat

,lang_index,lang,instanceId_userId
0,16,uz,1
1,13,sr,2
2,7,it,2
3,5,fr,14
4,4,et,15
5,6,hy,15
6,11,ro,24
7,8,ky,35
8,15,uk,43
9,10,ms,45


In [ ]:
'''audit_timestamp = train_data.audit_timestamp.apply(lambda x: int(x) // 1000)
plt.figure(figsize=(35,15))
plt.hist(audit_timestamp, 1800, color='red', alpha=0.5, stacked=True)
plt.show()'''

In [ ]:
'''plt.figure(figsize=(35,15))
plt.hist(audit_timestamp[(audit_timestamp % (3600*24) > 7*3600) & (audit_timestamp % (3600*24) < 20*3600)], 1800, color='red', alpha=0.5, stacked=True)
plt.show()'''

In [ ]:
def is_day(ts):
    ts = int(ts) // 1000
    x = ts % 3600*24
    return x > 7*3600 and x < 20*3600

train_data['is_day'] = train_data.metadata_createdAt.apply(is_day)
test_data['is_day'] = test_data.metadata_createdAt.apply(is_day)